In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as imgg
import os
import pandas as pd
import numpy as np

from PIL import Image
from PIL import ImageDraw

In [23]:
class_={
    0: 'ignored regions',
    1: 'pedestrian',
    2: 'people',
    3: 'bicycle',
    4: 'car',
    5: 'van',
    6: 'truck',
    7: 'tricycle',
    8: 'awning-tricycle',
    9: 'bus',
    10: 'motor', 
    11: 'others'
}

In [2]:
def read_annot(path='VisDrone2019-DET-train/annotations/0000002_00005_d_0000014.txt'):
    
    # process annotation files, return pandas dataframe
    
    annot=pd.read_csv(path,header=None)
    annot.columns=['x','y','w','h','conf','class','truncation','occlusion']
    
    annot['x_']=annot['x']+annot['w']
    annot['y_']=annot['y']+annot['h']
    
    return annot

In [3]:
def draw_boxes(image,boxes):
    
    # draw all given boxes onto image
    # boxes is list of tuples of (x,y,x_,y_)
    
    rec=Image.new('RGBA',image.size,(255,255,255,0))
    draw=ImageDraw.Draw(rec)
    
    for box in boxes: 
        draw.rectangle(box,outline=(255,255,0)) # yellow outline
        
    out=Image.alpha_composite(image.convert('RGBA'),rec)
    
    return out

In [4]:
def resized_box(image,box,new_size=(128,128),as_numpy=True):
    
    # from box, crop image and resize to desired size
    # box is tuple of (x,y,x_,y_)
    
    box_=image.crop(box)
    box_=box_.resize(new_size)
    
    if as_numpy:
        return np.array(box_)
    else:
        return box_
    

#### Demo, using VisDrone trainset

get from https://github.com/VisDrone/VisDrone-Dataset

In [13]:
im=Image.open('VisDrone2019-DET-train/images/0000002_00005_d_0000014.jpg')
im.show()

In [17]:
annot=read_annot('VisDrone2019-DET-train/annotations/0000002_00005_d_0000014.txt')
annot.head()

,x,y,w,h,conf,class,truncation,occlusion,x_,y_
0,684,8,273,116,0,0,0,0,957,124
1,406,119,265,70,0,0,0,0,671,189
2,255,22,119,128,0,0,0,0,374,150
3,1,3,209,78,0,0,0,0,210,81
4,708,471,74,33,1,4,0,1,782,504


In [18]:
l=[]
for i in range(annot.shape[0]):
    l.append(tuple(annot[['x','y','x_','y_']].iloc[i]))

In [19]:
boxed=draw_boxes(im,l)

In [20]:
boxed.show()

In [21]:
cars=[]
crs=annot[annot['class']==4]
for i in range(crs.shape[0]):
    cars.append(tuple(crs[['x','y','x_','y_']].iloc[i]))
    

In [22]:
boxed_cars=draw_boxes(im,cars)
boxed_cars.show()

### Keras models used like

In [ ]:
model=tf.keras.applications.VGG16()

In [ ]:
model.summary()

In [ ]:
help(tf.keras.applications.vgg16.preprocess_input)

In [ ]:
# works on numpy arrays!

pp_img=tf.keras.applications.vgg16.preprocess_input(img)

In [ ]:
model.predict(pp_img)

In [ ]:
tf.keras.applications.vgg16.decode_predictions(np.array(model(ppp_img.reshape((1,224,224,3)))))